In [1]:
import sys
sys.path.append('..')
sys.path.append('/Dataset')
import numpy as np
import pandas as pd
import os

from DressipiChallenge.Pipeline.submission import create_submission
from DressipiChallenge.Pipeline.utils import get_mapped_sessions_to_recommend, get_items_to_exclude, generate_predictions, create_mapping, single_train_test_model
from DressipiChallenge.Pipeline.matrices_creation import get_URM_split_val, get_URM_split_test
from DressipiChallenge.Pipeline.data_extraction import get_dataframes
from DressipiChallenge.Pipeline.recommender_tuning import hypertune
from DressipiChallenge.Pipeline.data_splitting import split_dataframes_val
from DressipiChallenge.Pipeline.optuna_utils import Range,Integer,Real,Categorical
from DressipiChallenge.Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from DressipiChallenge.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from DressipiChallenge.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from DressipiChallenge.Evaluation.Evaluator import EvaluatorHoldout

In [2]:
item_features_df, train_sessions_df, train_purchases_df, test_sessions_df, candidate_items_df = get_dataframes(path_to_dataset = '../')

CSVs read


In [3]:

URM_train, URM_val_views, URM_val_purch, mapped_items_to_ignore, mapped_val_sessions_arr, val_session_mapping, item_mapping = get_URM_split_val(
    item_features_df=item_features_df,
    train_purchases_df=train_purchases_df,
    train_sessions_df=train_sessions_df,
    val_start_ts = '2021-05-01',
    val_end_ts = '2021-06-01',
    abs_half_life=400,
    unique_interactions=True,
    cyclic_decay=True,
    purch_weight=1,
    view_weight=0.5,
    score_graph=False,
)

In [4]:
hyp = {'topK': None, 'normalize_matrix': False, 'l2_norm': 46.90311309040724}

In [5]:
single_train_test_model(EASE_R_Recommender, URM_train, URM_val_views, URM_val_purch, mapped_items_to_ignore, item_mapping, hyp)

EASE_R_Recommender: URM Detected 477 ( 2.0%) items with no interactions.
EvaluatorHoldout: Ignoring 18922 Items
EASE_R_Recommender: Fitting model... 
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 23691 (100.0%), 91.87 column/sec. Elapsed time 4.30 min
EASE_R_Recommender: Fitting model... done in 10.03 min
EvaluatorHoldout: Processed 81618 (100.0%) in 1.26 min. Users per second: 1080


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 100     0.004899                 0.489892  0.489892  0.001537    0.153681   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 100     0.153681  0.220318  0.009701  0.489892      0.153681  ...   
 
        COVERAGE_ITEM_CORRECT COVERAGE_USER COVERAGE_USER_CORRECT  \
 cutoff                                                             
 100                 0.633466           1.0              0.489892   
 
        DIVERSITY_GINI SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL  \
 cutoff                                                             
 100          0.278876       10.727169                   0.999847   
 
        RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY  \
 cutoff                                                

In [ ]:
weighted_train_set_df, val_purch_df, val_views_df, train_session_mapping, val_session_mapping, item_mapping,\
           mapped_items_to_ignore, mapped_val_sessions_arr = split_dataframes_val(
    train_sessions_df, train_purchases_df, item_features_df,
    val_start_ts='2021-05-01', val_end_ts='2021-06-01',
    view_weight=1, purch_weight=1, abs_half_life=None, cyclic_decay=False, score_graph=False,
)

In [ ]:
hyp = {
    'topK' : Integer(50, 5000),
    'alpha' : Real(0,5),
    'normalize_similarity' : Categorical([True,False]),
}

In [ ]:
hypertune(
    URM_train=URM_train,
    URM_val_views=URM_val_views,
    URM_val_purch=URM_val_purch,
    item_mapping=item_mapping,
    mapped_items_to_ignore=mapped_items_to_ignore,
    num_trials=200,
    recommender_class=P3alphaRecommender,
    hyperparameters_dict=hyp
)

In [ ]:
hyp = {'topK': 156, 'alpha': 0.9867577596015882, 'beta': 0.3006127016015112, 'normalize_similarity': True}

rec = RP3betaRecommender(URM_train)
rec.fit(**hyp)
rec.set_URM_train(URM_val_views)

In [ ]:
eval = EvaluatorHoldout(URM_val_purch, [100], ignore_items=mapped_items_to_ignore)
eval.evaluateRecommender(rec)

In [19]:
URM_train, URM_test_views, mapped_items_to_ignore, mapped_test_sessions_arr, test_session_mapping, item_mapping = get_URM_split_test(
    train_purchases_df=train_purchases_df,
    test_sessions_df=test_sessions_df,
    train_sessions_df=train_sessions_df,
    candidate_items_df=candidate_items_df,
    item_features_df=item_features_df,
    unique_interactions=True,
    abs_half_life=None,
    cyclic_decay=False,
    purch_weight=1,
    view_weight=0.2,
    score_graph=False,
)

In [20]:
hyp = {'topK': 156, 'alpha': 0.9867577596015882, 'beta': 0.3006127016015112, 'normalize_similarity': True}

rec = RP3betaRecommender(URM_train)
rec.fit(**hyp)
rec.set_URM_train(URM_test_views)
rec.set_items_to_ignore(mapped_items_to_ignore)

RP3betaRecommender: URM Detected 73 ( 0.3%) items with no interactions.


In [21]:
dataframe_list = generate_predictions(
    models=[rec],
    session_ids=mapped_test_sessions_arr,
    add_item_score=False,
    cutoff = 100,
)

Recommending...


100%|██████████| 10/10 [00:38<00:00,  3.80s/it]


Done!


In [22]:
df = dataframe_list[0]
df

,session_id,item_id
0,0,2735
1,0,4520
2,0,22360
3,0,17349
4,0,1843
...,...,...
4999995,49999,12171
4999996,49999,8424
4999997,49999,4074
4999998,49999,10181


In [23]:
create_submission(
    reranked_df=df,
    item_mapping=item_mapping,
    session_mapping=test_session_mapping,
    save_path='./subs/RP3beta/'
)

SUBMISSION COMPLETED


# Step by step pipeline

In [ ]:
def time_weighting(df, ref_date, time_mode, abs_time_weight, f):

    if 'score' not in df.columns.to_list():
        df['score'] = 1

    if time_mode == 1:
        df['abs_delta'] = [(ref_date-date).days  for date in df.date]
        df['score'] *= round(np.exp(-abs_time_weight*df.abs_delta), 4)
    elif time_mode == 2:
        df['abs_delta'] = [(ref_date-date).days  for date in df.date]
        df['score'] *= round( (1 + np.cos(2 * np.pi * f * df.abs_delta)) / 2, 4)
    elif time_mode == 3:
        df['abs_delta'] = [(ref_date-date).days  for date in df.date]
        df['score_1'] = round(2 + np.cos(2 * np.pi * f * df.abs_delta), 4)
        df['score_2'] = round(np.exp(-abs_time_weight*df.abs_delta), 4)
        df['score'] *= round(df['score_1'] * df['score_2'] / 3, 4)
        df.drop(columns=['score_1', 'score_2'], inplace=True)
    
    return df

In [ ]:
view_weight = 0.5
purch_weight = 1

In [ ]:
val_start_ts='2021-05-01'
val_end_ts='2021-06-01'

pd.options.mode.chained_assignment = None

sorted_sessions_df = train_sessions_df.sort_values(['date'], ascending=True)
sorted_sessions_df.drop_duplicates(['session_id'], keep='first', inplace=True) # DUPLICATE VIEWS ARE DISCARDED!

train_sessions = sorted_sessions_df[sorted_sessions_df['date'] < val_start_ts]['session_id'].values
val_sessions = sorted_sessions_df[(sorted_sessions_df['date'] >= val_start_ts) & (sorted_sessions_df['date'] < val_end_ts)]['session_id'].values

selected_train_views_df = train_sessions_df[train_sessions_df['session_id'].isin(train_sessions)]
selected_train_purchases_df = train_purchases_df[train_purchases_df['session_id'].isin(train_sessions)]

selected_train_views_df['score'] = view_weight
selected_train_purchases_df['score'] = purch_weight

train_set_df = pd.concat([selected_train_views_df, selected_train_purchases_df]).sort_values(['session_id', 'date'], ascending=[True, True]).reset_index(drop=True)

val_set_df = train_purchases_df[train_purchases_df['session_id'].isin(val_sessions)].reset_index(drop=True)

In [ ]:
train_set_df

In [ ]:
train_session_mapping = create_mapping(train_set_df['session_id'])
val_session_mapping = create_mapping(val_set_df['session_id'])
item_mapping = create_mapping(item_features_df['item_id'])

val_sessions_arr = get_mapped_sessions_to_recommend(val_set_df, val_session_mapping)

In [ ]:
train_set_df['session_id'] = train_set_df['session_id'].map(train_session_mapping)
train_set_df['item_id'] = train_set_df['item_id'].map(item_mapping)

val_set_df['session_id'] = val_set_df['session_id'].map(val_session_mapping)
val_set_df['item_id'] = val_set_df['item_id'].map(item_mapping)

recommendable_items = np.unique(val_set_df['item_id'].values)
items_to_ignore = get_items_to_exclude(item_features_df, recommendable_items)

In [ ]:
abs_half_life = 200
cyclic_decay = True
frequency = 1/365

In [ ]:
time_mode = 0
abs_time_weight = 0
if abs_half_life is not None:
    time_mode+= 1
    abs_time_weight = 1/abs_half_life
if cyclic_decay:
    time_mode+= 2

train_set_df['date'] = pd.to_datetime(train_set_df['date'])
weighted_train_set_df = time_weighting(train_set_df, pd.to_datetime(val_start_ts), time_mode, abs_time_weight, frequency)

In [ ]:
score_graph = 1
import matplotlib.pyplot as plt
if score_graph:    
    weighted_train_set_df.plot(x='date', y='score')
    plt.show()

In [ ]:
import scipy.sparse as sp

URM = sp.csr_matrix((weighted_train_set_df['score'].values,
                        (weighted_train_set_df['session_id'].values, weighted_train_set_df['item_id'].values)),
                    shape=(len(train_session_mapping), len(item_mapping)))
URM